In [2]:
# Import libraries
import pandas as pd
import sys
import os

In [3]:
# Get the current working directory
current_dir = os.getcwd()
# Move up one level from the current directory
parent_dir = os.path.dirname(current_dir)
# Change directory into data directory
data_dir = os.path.join(parent_dir, 'data')

In [4]:
sys.path.append(parent_dir)

In [5]:
#List all files in the parent directory
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f'{subindent}{f}')

# Use '..' to refer to the parent directory
list_files('..')

../
    LICENSE
    requirements.txt
    README.md
    .gitignore
    package-lock.json
    package.json
    app.py
    db_config.py
    tabs/
        recommendation.py
        menu.py
        recipe.py
        __pycache__/
            recommendation.cpython-311.pyc
            menu.cpython-312.pyc
            recipe.cpython-311.pyc
            menu.cpython-311.pyc
    images/
        wines2.jpeg
        wines.jpg
        logo.png
    html/
        univariate.html
    __pycache__/
        db_config.cpython-312.pyc
        db_config.cpython-311.pyc
        config.cpython-311.pyc
    model/
        wine_rec.py
        __pycache__/
            wine_rec.cpython-311.pyc
            wine_rec.cpython-312.pyc
    venv/
        pyvenv.cfg
        bin/
            streamlit.cmd
            pyftsubset
            jupyter-run
            Activate.ps1
            streamlit
            dotenv
            pip-chill
            python3
            pip3.12
            ttx
            python
           

In [6]:
import db_config as cfg

# 1. Extract

In [7]:
df_product = pd.read_csv(os.path.join(data_dir, 'products.csv'), index_col=None)

In [8]:
df_product.head()

,item_desc,corp_item_brand_name,pim_item_class_desc,pim_item_sub_class_desc,state,flavor,pim_tasting_notes,alcohol_percentage,sweetness_level,bitterness_level,...,body,serving_temperature,vintage_year,grape_variety,region,price,food_pairing,aroma,mouthfeel,finish
0,-196 CKTL VOD A(DL/G/P) CAN 3/8PK,-196,COCKTAILS,COCKTAILS-OTHER,TX,OTHER,NaN,12.437938,8,10,...,Light,47.499006,2019,Chardonnay,Sonoma,76.968522,Pasta,Fruity,Silky,Short
1,-196 CKTL VOD PEACH 12 CAN 6/4PK,-196,COCKTAILS,COCKTAILS-OTHER,TX,PEACH,NaN,12.995617,8,3,...,Light,43.392326,2005,Chardonnay,Sonoma,21.302044,Grilled Chicken,Fruity,Silky,Long
2,-196 CKTL VOD DBL LEM 12 CAN 6/4PK,-196,COCKTAILS,COCKTAILS-OTHER,TX,LEMON,Fresh lemon peel. Tart and light sweet with ch...,11.234420,7,6,...,Medium,57.216822,2017,Merlot,Sonoma,23.523335,Cheese,Floral,Velvety,Long
3,-196 CKTL VOD GRFRUIT 12 CAN 6/4P,-196,COCKTAILS,COCKTAILS-OTHER,TX,GRAPEFRUIT,NaN,5.381421,4,4,...,Full,47.878084,2007,Cabernet Sauvignon,Tuscany,67.868299,Pasta,Fruity,Velvety,Short
4,10 CANE RUM 80,10 CANE,RUM,GOLD RUM,TX,NaN,NaN,13.381235,7,4,...,Full,51.834639,2008,Cabernet Sauvignon,Tuscany,95.581735,Pasta,Fruity,Silky,Short


# 2. Transform

In [9]:
df_product.dtypes

item_desc                   object
corp_item_brand_name        object
pim_item_class_desc         object
pim_item_sub_class_desc     object
state                       object
flavor                      object
pim_tasting_notes           object
alcohol_percentage         float64
sweetness_level              int64
bitterness_level             int64
acidity_level                int64
tannin_level                 int64
body                        object
serving_temperature        float64
vintage_year                 int64
grape_variety               object
region                      object
price                      float64
food_pairing                object
aroma                       object
mouthfeel                   object
finish                      object
dtype: object

# 3. Load

In [10]:
import pyodbc
import psycopg2

In [11]:
def get_column_types(df):
    # Map Pandas data types to SQL Server data types
    type_map = {
        'object': 'NVARCHAR(MAX)',
        'int64': 'BIGINT',
        'float64': 'DECIMAL(18, 5)',  # Use DECIMAL with precision and scale
        'datetime64[ns]': 'DATETIME2',
        'bool': 'BIT'
    }
    return [type_map.get(str(dt), 'NVARCHAR(MAX)') for dt in df.dtypes]


In [15]:
import psycopg2
from psycopg2 import sql
import db_config as cfg

def test_postgres_connection():
    # Connection parameters
    conn_params = {
        'host': cfg.HOST,
        'port': cfg.PORT,
        'database': cfg.DATABASE_NAME,
        'user': cfg.USER_ID,
        'password': cfg.USER_PASSWORD
    }

    try:
        # Establish a connection
        conn = psycopg2.connect(**conn_params)
        cursor = conn.cursor()

        # Create the 'dbo' schema if it doesn't exist
        cursor.execute("CREATE SCHEMA IF NOT EXISTS dbo")

        # Create a test table named 'testPost' in the 'dbo' schema
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS dbo.testPost (
                id SERIAL PRIMARY KEY,
                name VARCHAR(50),
                value INTEGER
            )
        """)

        # Insert sample data
        cursor.execute("""
            INSERT INTO dbo.testPost (name, value) VALUES
            ('Alice', 100),
            ('Bob', 200),
            ('Charlie', 300)
        """)

        # Commit the changes
        conn.commit()

        # Fetch and print the data
        cursor.execute("SELECT * FROM dbo.testPost")
        rows = cursor.fetchall()

        print("Connection successful!")
        print("Sample data inserted and retrieved from dbo.testPost:")
        for row in rows:
            print(f"ID: {row[0]}, Name: {row[1]}, Value: {row[2]}")

    except psycopg2.Error as e:
        print(f"An error occurred: {e}")

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Run the test function
test_postgres_connection()

Connection successful!
Sample data inserted and retrieved from dbo.testPost:
ID: 1, Name: Alice, Value: 100
ID: 2, Name: Bob, Value: 200
ID: 3, Name: Charlie, Value: 300


In [32]:
def load_table(df, table_name, schema_name):
    import time
    start_time = time.time()
    
    # Define the connection parameters
    conn_params = {
        'host': cfg.HOST,
        'port': cfg.PORT,
        'database': cfg.DATABASE_NAME,
        'user': cfg.USER_ID,
        'password': cfg.USER_PASSWORD
    }

    try:
        # Create a psycopg2 connection using the connection parameters
        conn = psycopg2.connect(**conn_params)
        cursor = conn.cursor()

        # Drop the table if it already exists
        cursor.execute(f"DROP TABLE IF EXISTS {schema_name}.{table_name}")
        conn.commit()

        # Create the table schema
        def get_postgres_column_types(df):
            type_map = {
                'object': 'TEXT',
                'int64': 'BIGINT',
                'float64': 'DECIMAL(18, 5)',
                'datetime64[ns]': 'TIMESTAMP',
                'bool': 'BOOLEAN'
            }
            return [type_map.get(str(dt), 'TEXT') for dt in df.dtypes]

        columns = ', '.join([f'"{col}" {dtype}' for col, dtype in zip(df.columns, get_postgres_column_types(df))])
        create_table_sql = f'CREATE TABLE {schema_name}.{table_name} ({columns})'
        cursor.execute(create_table_sql)
        conn.commit()

        # Clean up the DataFrame
        df = df.copy()  # Create a copy to avoid SettingWithCopyWarning
        for col in df.select_dtypes(include=['float']):
            df[col] = df[col].round(5)  # Round to 5 decimal places

        # Handle NaN and None values
        df = df.fillna(value={col: 0 if df[col].dtype.kind in 'biufc' else '' for col in df.columns})

        # Insert the data into the table in smaller batches
        batch_size = 100  # Reduced batch size for faster processing
        for i in range(0, len(df), batch_size):
            batch = df.iloc[i:i+batch_size]
            insert_sql = f"INSERT INTO {schema_name}.{table_name} ({','.join([f'\"{col}\"' for col in batch.columns])}) VALUES ({','.join(['%s'] * len(batch.columns))})"
            cursor.executemany(insert_sql, batch.values.tolist())
            conn.commit()
            print(f"Inserted batch {i//batch_size + 1} of {(len(df)-1)//batch_size + 1}")

        print(f"DataFrame loaded successfully into {schema_name}.{table_name}")
        print(f"Total time taken: {time.time() - start_time:.2f} seconds")
    except psycopg2.Error as e:
        print(f"A database error occurred: {e}")
        conn.rollback()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

In [35]:
load_table(df_product, 'products', 'dbo')

Inserted batch 1 of 178
Inserted batch 2 of 178
Inserted batch 3 of 178
Inserted batch 4 of 178
Inserted batch 5 of 178
Inserted batch 6 of 178
Inserted batch 7 of 178
Inserted batch 8 of 178
Inserted batch 9 of 178
Inserted batch 10 of 178
Inserted batch 11 of 178
Inserted batch 12 of 178
Inserted batch 13 of 178
Inserted batch 14 of 178
Inserted batch 15 of 178
Inserted batch 16 of 178
Inserted batch 17 of 178
Inserted batch 18 of 178
Inserted batch 19 of 178
Inserted batch 20 of 178
Inserted batch 21 of 178
Inserted batch 22 of 178
Inserted batch 23 of 178
Inserted batch 24 of 178
Inserted batch 25 of 178
Inserted batch 26 of 178
Inserted batch 27 of 178
Inserted batch 28 of 178
Inserted batch 29 of 178
Inserted batch 30 of 178
Inserted batch 31 of 178
Inserted batch 32 of 178
Inserted batch 33 of 178
Inserted batch 34 of 178
Inserted batch 35 of 178
Inserted batch 36 of 178
Inserted batch 37 of 178
Inserted batch 38 of 178
Inserted batch 39 of 178
Inserted batch 40 of 178
Inserted 